In [58]:
from nltk.sentiment import SentimentIntensityAnalyzer
import plotly.graph_objects as go
import pandas as pd
import nltk

from plotly.subplots import make_subplots

import s3
first = s3.extract('dev/Top_Lecture/', 'SON', 'GPT')
second = s3.extract('dev/Other_Lecture/', 'BYUN', 'GPT')
user = s3.extract('user/transcript/', '20231219173149')

sia = SentimentIntensityAnalyzer()

sentences_top = nltk.sent_tokenize(first)
positive = 0
negative = 0
neutral = 0
for sentence in sentences_top:
    sentiment = sia.polarity_scores(sentence)
    if sentiment['compound'] > 0.05:
        positive += 1
    elif sentiment['compound'] < -0.05:
        negative += 1
    else:
        neutral += 1
df_top_sent = pd.DataFrame({'sentiment': ['positive', 'negative', 'neutral'], 'count': [positive, negative, neutral]})

sentences_bottom = nltk.sent_tokenize(second)
positive_bottom = 0
negative_bottom = 0
neutral_bottom = 0
for sentence in sentences_bottom:
    sentiment = sia.polarity_scores(sentence)
    if sentiment['compound'] > 0.05:
        positive_bottom += 1
    elif sentiment['compound'] < -0.05:
        negative_bottom += 1
    else:
        neutral_bottom += 1
df_other_sent = pd.DataFrame({'sentiment': ['positive', 'negative', 'neutral'], 'count': [positive_bottom, negative_bottom, neutral_bottom]})

sentences_bottom = nltk.sent_tokenize(user)
positive_bottom = 0
negative_bottom = 0
neutral_bottom = 0
for sentence in sentences_bottom:
    sentiment = sia.polarity_scores(sentence)
    if sentiment['compound'] > 0.05:
        positive_bottom += 1
    elif sentiment['compound'] < -0.05:
        negative_bottom += 1
    else:
        neutral_bottom += 1
df_user_sent = pd.DataFrame({'sentiment': ['positive', 'negative', 'neutral'], 'count': [positive_bottom, negative_bottom, neutral_bottom]})

fig = go.Figure()
fig = make_subplots(rows=1, cols=3, specs=[[{'type':'domain'}, {'type':'domain'}, {'type':'domain'}]], subplot_titles=['Top Lecture Sentences', 'Other Lecture Sentences', 'Your Sentences'])
fig.add_trace(go.Pie(labels=df_top_sent['sentiment'], 
                     values=df_top_sent['count'], 
                     hole=.3,), row=1, col=1)
fig.add_trace(go.Pie(labels=df_other_sent['sentiment'], 
                     values=df_other_sent['count'], 
                     hole=.3,
                     ), row=1, col=2)
fig.add_trace(go.Pie(labels=df_user_sent['sentiment'], 
                     values=df_user_sent['count'], 
                     hole=.3), row=1, col=3)

fig.update_layout(title_text="<b>Sentence Sentimental Analysis<b>",
                  )
# fig.update_xaxes(title_text="")
# fig.update_yaxes(title_text="")
fig.show()

<<<<< Txt List >>>>>
['dev/Top_Lecture/중학뉴런수학2학년(상)_SON_GPT_41672.txt']
<<<<< Txt List >>>>>
['dev/Other_Lecture/중학뉴런수학2학년(상)_BYUN_GPT_62031.txt']
<<<<< Txt List >>>>>
['user/transcript/20231219173149_transcript.txt']


In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import euclidean
import plotly.graph_objects as go

import s3
ncic = s3.extract('dev/NCIC/', 'NCIC')
first = s3.extract('dev/Top_Lecture/', 'SON', 'GPT')
second = s3.extract('dev/Other_Lecture/', 'BYUN', 'GPT')
user = s3.extract('user/transcript/', '20231219173149')

# Convert the texts into TF-IDF features
vectorizer = TfidfVectorizer().fit_transform([ncic, first, second, user])
vectors = vectorizer.toarray()
# Compute the cosine similarity
cosine_sim = cosine_similarity(vectors)
# Compute the Euclidean distance
euclidean_distance_ncic_first = euclidean(vectors[0], vectors[1])
euclidean_distance_ncic_second = euclidean(vectors[0], vectors[2])
euclidean_distance_ncic_user = euclidean(vectors[0], vectors[3])
# Plot the results
labels = ['NCIC-Top_Lecture', 'NCIC-Other_Lecture', 'NCIC-User']
cosine_values = [cosine_sim[0][1], cosine_sim[0][2], cosine_sim[0][3]]
euclidean_values = [euclidean_distance_ncic_first, euclidean_distance_ncic_second, euclidean_distance_ncic_user]
fig = go.Figure(data=[
    go.Bar(name='Cosine Similarity', x=labels, y=cosine_values),
    go.Bar(name='Euclidean Distance', x=labels, y=euclidean_values)
])
# Change the bar mode
fig.update_layout(barmode='group', title_text='Comparison Between NCIC, Top Rank Lecture and Other Rank Lecture and Your Video')
fig.show()

<<<<< Txt List >>>>>
['dev/NCIC/NCIC_ALL_GPT_KOR_957.txt', 'dev/NCIC/수학목표_중학수학성취기준.txt']
<<<<< Txt List >>>>>
['dev/Top_Lecture/중학뉴런수학2학년(상)_SON_GPT_41672.txt']
<<<<< Txt List >>>>>
['dev/Other_Lecture/중학뉴런수학2학년(상)_BYUN_GPT_62031.txt']
<<<<< Txt List >>>>>
['user/transcript/20231219173149_transcript.txt']
